In [ ]:
%matplotlib inline
import numpy as np
from numpy import average, hstack, argsort, ones, sort, diff, where

from ipywidgets import widgets

from itertools import cycle
import lmfit
from scipy.io import savemat, loadmat
from matplotlib import pylab as pl
import scipy.optimize as sciop
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

from scipy.ndimage.measurements import label, find_objects, center_of_mass
from scipy.ndimage.filters import sobel, gaussian_filter1d, median_filter, laplace
from scipy.signal import find_peaks_cwt

from scipy.interpolate import UnivariateSpline, interp1d

import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot,iplot_mpl
import plotly.graph_objs as go
init_notebook_mode()

c_light = 3e8

Kippemberg paper
[doi:10.1038/nphoton.2009.138]

In [ ]:
# def get_peaks(t, c, min_dist = 0.0000035):
    ## isolate each peak
    #l, n = label(c)
    #peaks = np.array([average(t[obj]) for obj in find_objects(l)])
    #return peaks

fit_max_f = lmfit.Model(lambda x, x0, a, g: a/(1+(x-x0)**2/(g)**2))

def fit_max(x,y):
    #fit_r = fit_max_f.fit(data=y, x = x, x0 = x[np.argmax(y)],
      #                    a = np.amax(y), g = 100)
    return x[np.argmax(y)]
    return fit_r.params['x0'].value

def LorentzianLin(m,b,amp,xc,gam,x):
    outs=m*(x-xc)+b+(amp/(np.pi*2))/((x-xc)**2+(gam/2)**2)
    return outs

def Lorentzian(b,amp,xc,gam,x):
    outs=b+(amp/(np.pi*2))/((x-xc)**2+(gam/2)**2)
    return outs

def Quadratic(a,b,c,x):
    outs=a+b*x+c*x*x
    return outs

def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or np.fabs(value - array[idx-1]) < np.fabs(value - array[idx])):
        return idx-1
    else:
        return idx
    
def rollingWindow(a, window, edge = 'copy'):
    if edge == 'copy':
        extended = np.zeros(a.shape[:-1] + (a.shape[-1]+window-1, ))
        extended[..., int(window/2.-1): -int(window/2.)] = a[...]
        extended[..., :int(window/2.+1)]= a[...,0]
        extended[..., -int(window/2.):]= a[...,-1]
        a = extended
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def rollingAverage(a,win_size=10):
    out=average(rollingWindow(a, win_size), -1)
    #out1=np.append(np.zeros(int(window/2)),out)
    #out2=out1[:-int(window/2)] #these two modiciations make the filter symetric
    return out

import numpy as np
def find_nearest_unsrt(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [ ]:
filename='rb_1555_1570_200nm_1'
#filename = 'test_pratham_1530_1600'
#filename = 'scan_1530_1570_85sec_10_4_16'
#filename = 'scan_1540_1570_85sec_10_4_16_agilent'
#trans = loadmat('C:\\Users\\Rees\\Documents\\Dispersion Data\\'+filename+'_tr', squeeze_me = True)['data']
trans = loadmat(filename+'_tr', squeeze_me = True)['data']
ind = np.where(~np.isnan(trans[1]))[0]
first, last = ind[0], ind[-1]
trans[1][:first] = trans[1][first]
trans[1][last + 1:] = trans[1][last]

In [ ]:
# ## smooths, and removes a low-passed background, on a subset of data to test settings
highpassbin=20000
lowpassbin=400
# testx=trans[0,100000:1000000]
# testy=trans[1][100000:1000000]
# trans_ptest = rollingAverage(testy,lowpassbin)-rollingAverage(testy,highpassbin)
# trans_ptest = np.nan_to_num(trans_ptest)
# f,ax = plt.subplots(1,1, figsize=(8,4))
# ax.plot(testx, trans_ptest)
# ax.plot(testx, testy-np.mean(testy))
# iplot_mpl(f)

In [ ]:
## smooths, and saves the full data set, to avoid having to filter it again
trans_p = rollingAverage(trans[1],lowpassbin)-rollingAverage(trans[1],highpassbin)
trans[1]=trans_p
savemat(filename+'_tr'+'_filt', {'data': np.vstack([trans[0], trans[1]])}) 

In [ ]:
## Loads the smoothed dataset, to avoid re-filtering
trans = loadmat(filename+'_tr_filt', squeeze_me = True)['data']
ind = np.where(~np.isnan(trans[1]))[0]
first, last = ind[0], ind[-1]
trans[1][:first] = trans[1][first]
trans[1][last + 1:] = trans[1][last]

In [ ]:
## Plots the smoothed data, to enable manual peak identification
f,ax = pl.subplots(1,1, figsize=(10,4))
ax.set_xlim(0,1e7)
line=ax.plot(trans[0,::25],trans[1,::25])
iplot_mpl(f)
savemat(filename+'_tr'+'_filtered_spectrum', {'peaks1':trans[0],'peaks2':trans[1]}),

Now, use the the plot above, to zoom into the first set of nice looking resonances you see. If you have multiple resonance you wish to track the format is

ModePeak=[[Peak1(0),Peak1(1)],[Peak2(0),Peak2(1)],[...]]

The program will then take these initial peaks, and hunt for the rest of the set of peaks based on the seperation of the initial peaks and their location. It does this by guessing where the next will be by extrapolating the previous few peaks, looking at "fitsize" points around the guess, and finding the minum value inside that small search window.

Paramters to adjust are the "fitsize", and "peakstofit" which sets how many peaks it will look for in each mode family before it exits 

In [ ]:
ModePeak=[[.1420803,.2472956],[1,2]]

ModePeak=np.multiply(ModePeak,1E6)
fitsize=8000 #number of points around guess to fit
modefamilias=1#number of distinct sets of peaks you want to fit
peakstofit=20
peaks=[[],[],[]]

f,ax = pl.subplots(1,1, figsize=(10,4))
for j in range(modefamilias):
    modeselect=j
    indexmin = find_nearest(trans[0],ModePeak[modeselect][0])
    datx=trans[0][indexmin-fitsize:indexmin+fitsize]
    daty=trans[1][indexmin-fitsize:indexmin+fitsize]                                                                                                        
    start=datx[find_nearest_unsrt(daty,min(daty))]
    PeakLoc=[start]
    ax.plot(datx,daty)
    gap=ModePeak[modeselect][1]-start
    locguess=start+gap
    indices=[]
    for i in range(peakstofit):
        indices.append(i)
        indexmin = find_nearest(trans[0],locguess)
        datx=trans[0][indexmin-fitsize:indexmin+fitsize]
        daty=trans[1][indexmin-fitsize:indexmin+fitsize]
        thismin=datx[find_nearest_unsrt(daty,min(daty))]
        if i>10:
            PeakLoc.append(thismin) 
            xdat=[0,1,2,3,4,5,6]
            ydat=PeakLoc[-7:]
            fits=np.poly1d(np.polyfit(xdat, ydat,2))
            locguess= fits(7)
        else:
            gap=thismin-PeakLoc[-1]
            PeakLoc.append(thismin) 
            locguess= thismin + gap
        ax.plot(datx[::50],daty[::50])
    peaks[j]=PeakLoc
iplot_mpl(f)

Here, you can see how each fitted peak looks. You can use this to diagnoise issues, and see if "fitsize" is properly set.

The next block, shows an example of the FSR versus frequency of the sweep to diagnose issues

In [ ]:
start=0
cut=80
fsr1=np.gradient(np.asarray(peaks[0][start:cut]))
#fsr2=np.gradient(np.asarray(peaks[1][start:cut]))
#fsr3=np.diff(np.asarray(peaks[2][start:cut]))

f,ax = pl.subplots(1,1, figsize=(10,4))
ax.plot(peaks[0][0:cut],fsr1)  
#ax.plot(peaks[1][0:cut],fsr2)  
#ax.plot(fsr3)
iplot_mpl(f)

Finally, we save the locations of each of the fitted peaks. This needs to be modifed to reflect the number of mode families you fit
{'peaks1':peaks[0]}->{'peaks1':peaks[0],'peaks2':peaks[1],.....}

At this point, you will have the locations of many modes, and analysis is more convenient in a higher level programming language (aka MatLab)

Issues the data may have, is 250 Mhz jumps when a comb tooth is missed, and if you see consistently grouped peaks, with a few discrete disagreements, its totally fair to artifically adjust the 'bad' points by discrete amounts.

Typical performance as of 11/3/2016 is a 10 Mhz statistical uncertainty in fitted resonance locations, and an 8 Mhz statistical uncertainty in FSR, over a set of 10 measurements of the same chip.

In [ ]:
## Save transmission and frequency in one file
savemat(filename+'_tr'+'_fsr', {'peaks1':peaks[0],'peaks2':peaks[1]}),
                        #'options': dict(t_range = t_range,
                        #                dem_th = dem_th,
                        #                dem_alpha = dem_alpha,
                        #                do_patch = do_patch,
                        #                patch_pattern = patch_pattern)})

In [ ]:
# This is for '12_8_2016_Jae_doublering4_950_1400_200GHZ_XXX' data series
#ModePeak=[[.1576652,.3851763],[.4093255,.6396964]] #bad data
#ModePeak=[[.1661502,.3777254],[1.949471,2.159452]] #no second mode family
#ModePeak=[[.1492225,.3608335],[.704079,.566845]]   #no secons mode family
#ModePeak=[[.1969874,.4095016],[.21938,.4308126]]
#ModePeak=[[.2075429,.4190832],[.4419086,.6528858]]
#ModePeak=[[.1957864,.4073583],[.4448152,.6561047]]
#ModePeak=[[.1683048,.3798743],[.704079,.566845]]   #no second mode family
#ModePeak=[[.1786431,.3902426],[.21938,.4308126]]
#ModePeak=[[.158146,.3697491],[.1668186,.3817283]]
#ModePeak=[[.206704,.4182609],[.2153452,.4300555]]
#ModePeak=[[.1643852,.3759786],[.1730335,.3878131]]
#ModePeak=[[.3972519,.6085367],[.4132746,.6239653]]
#odePeak=[[.01818814,.229771],[.03208237,.2433376]]
#ModePeak=[[.194493309,.4060884],[.704079,.566845]]
#ModePeak=[[.2018233,.4134242],[.704079,.566845]]


# for the '1530_1610_r3_ctl_Hpol_11_12_2016_xxx' set of data
#ModePeak=[[.030737,.1251096],[.4704079,.566845]]
#ModePeak=[[.057018,.1514607],[.4704079,.566845]]
#ModePeak=[[.06670165,.1611261],[.4704079,.566845]]
#ModePeak=[[.0642822,.158566],[.4704079,.566845]]
#ModePeak=[[.06211067,.1565112],[.4704079,.566845]]
#ModePeak=[[.01329404,.1076933],[.4704079,.566845]]
#ModePeak=[[.03531843,.12974],[.4704079,.566845]]
#ModePeak=[[.0864544,.1808618],[.4704079,.566845]]

# Enter two adjacent locations, for each identifiable mode [[a1,a2],[b1,b2],[c1,c2]]
#ModePeak=[[2.315067,2.410103],[2.351455,2.445791],[2.355071,2.451556]] #pratham_2 initial guesses
#ModePeak=[[.3384759,.4329465],[.3903086,.4867791]]#10_20_2016_Agilent_thermal_on_1530_1570_1 initial guess
#ModePeak=[[.4106326,.5049396],[.4643857,.5608583]] #10_20_2016_Agilent_thermal_off_1530_1570_1 initial guess
#ModePeak=[[.009349,.1038205],[.0547619,.1512781]] #10_20_2016_Agilent_thermal_off_1530_1570_2 initial guess
#ModePeak=[[.3466385,.4411104],[.398475,.4949343]] #10_20_2016_Agilent_thermal_on_1530_1570_2 initial guess

#ModePeak=[[.2082301,.3026331],[.2599713,.3564166]] #'10_25_2016_CTL_thermal_off_1530_1610_1'
#ModePeak=[[.0446599,.1391195],[1,2]] #'10_25_2016_CTL_thermal_off_1530_1610_2'
#ModePeak=[[.2010087,.295358],[1.60369,1.700163]] #'10_25_2016_CTL_thermal_off_1530_1610_3'
#ModePeak=[[.03547385,.1298593],[1,2]]#'10_25_2016_CTL_thermal_off_1530_1610_4'
#ModePeak = [[2.281411,2.376414],[1,2]] #'10_25_2016_CTL_thermal_off_1530_1610_5'

#for the 10_27_2016_CTL_thermal_off_1530_1610 series of data, in order
#ModePeak=[[.01278,.1070939],[1,2]]
#ModePeak=[[.04172128,.136025],[1,2]]
#ModePeak=[[.07201787,.1664154],[1,2]]
#ModePeak=[[.02944836,.123748],[1,2]]
#ModePeak=[[.07240215,.1667194],[1,2]]
#ModePeak=[[.07012679,.1645023],[1,2]]
#ModePeak=[[.037147,.131462],[1,2]]
#ModePeak=[[.04735518,.141726],[1,2]]
#ModePeak=[[.02382155,.1181836],[1,2]]
#ModePeak=[[.0390736,.133407],[1,2]]

#for the 10_27_2016_CTL_thermal_off_1530_1610 series of data, in order
#ModePeak=[[.01278,.1070939],[.4441082,.5405605]]
#ModePeak=[[.04172128,.136025],[.473012,.569514]]
#ModePeak=[[.07201787,.1664154],[.4089817,.505475]]
#ModePeak=[[.02944836,.123748],[.4607695,.5572036]]
#ModePeak=[[.07240215,.1667194],[.4072247,.5037364]]
#ModePeak=[[.07012679,.1645023],[.4049401,.5014603]]
#ModePeak=[[.037147,.131462],[.4684524,.5649]]
#ModePeak=[[.04735518,.141726],[.4786845,.5751437]]
#ModePeak=[[.02382155,.1181836],[.4554438,.5518753]]
#ModePeak=[[.0390736,.133407],[.4704079,.566845]]



#for the 11_10_2016_CTL_1530_1610_ series of data, in collection order
#ModePeak=[[.0945437,.2924154],[.4704079,.566845]]
#ModePeak=[[.1800357,.3780944],[.4704079,.566845]]
#ModePeak=[[.02699714,.2252697],[.4704079,.566845]]
#ModePeak=[[.08198492,.2802272],[.4704079,.566845]] #Still bad for some reason
#ModePeak=[[.04347161,.2417198],[.4704079,.566845]]
#ModePeak=[[.0729834,.2712292],[.4704079,.566845]]
#ModePeak=[[.1122395,.3105037],[.4704079,.566845]]
#ModePeak=[[.06495976,.2632406],[.4704079,.566845]]
#ModePeak=[[.02021572,.2184811],[.4704079,.566845]]
#ModePeak=[[.1637376,.3615994],[.4704079,.566845]]

# For the '11_29_2016_1530_1610_TE_' series of data, in collection order
# This data is alex's chip, with TE polarization
#ModePeak=[[.13051,.32875],[.4704079,.566845]]
#ModePeak=[[.06997713,.2682505],[.4704079,.566845]]
#ModePeak=[[.1172247,.3155148],[.4704079,.566845]]
#ModePeak=[[.1902402,.3885278],[.4704079,.566845]]
#ModePeak=[[.1267433,.3250211],[.4704079,.566845]]
# This data is alex's chip, with TM polarization
#ModePeak=[[.0335342,.230357],[.4704079,.566845]]

# This data is alex's chip, with TE polarization looking at higher order Mode
#ModePeak=[[1.102855,1.29276],[1.29276,.566845]]
# This data is alex's chip, with TM polarization Looking at higher order Mode
#ModePeak=[[1.20453,1.392146],[.4704079,.566845]]



# This is for '12_8_2016_Pratham_1530_1570_TE_type2a4r4_1' data series
#ModePeak=[[.2072209,.3007048],[.4093255,.6396964]] 
#ModePeak=[[.1640734,.2576819],[.4093255,.6396964]] 
#ModePeak=[[.05806241,.1515734],[.4093255,.6396964]] 
#ModePeak=[[.1393745,.232804],[.4093255,.6396964]] 
#ModePeak=[[.1228236,.2162741],[.4093255,.6396964]] 

# This is for '12_8_2016_Pratham_1530_1570_TE_type2a4r3_1' data series
#ModePeak=[[.07815829,.1720419],[.09987,.1899297]] 
#ModePeak=[[.09451445,.1880102],[.1158486,.2058542]] 
#ModePeak=[[.05480697,.1474464],[.07941727,.1691536]] 
#ModePeak=[[.1086339,.2017672],[.1300376,.2199785]] 
#ModePeak=[[.03391709,.1274601],[.0557347,.1457869]] 

# This is for '12_8_2016_Pratham_1530_1570_TE_type2a4r1_1' data series
#ModePeak=[[.2405592,.3303456],[.09987,.1899297]] 
#ModePeak=[[.0847509,.1745493],[.0557347,.1457869]] #is bad
#ModePeak=[[.0847509,.1745493],[.0557347,.1457869]] 
#ModePeak=[[.05812133,.1480766],[.0557347,.1457869]] 
#ModePeak=[[.02275551,.1127163],[.0557347,.1457869]]
#ModePeak=[[.04673739,.1366031],[.0557347,.1457869]] 


In [ ]:
"""Detect peaks in data based on their amplitude and other features."""

from __future__ import division, print_function
import numpy as np

__author__ = "Marcos Duarte, https://github.com/demotu/BMC"
__version__ = "1.0.4"
__license__ = "MIT"


def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):

    """Detect peaks in data based on their amplitude and other features.

    Parameters
    ----------
    x : 1D array_like
        data.
    mph : {None, number}, optional (default = None)
        detect peaks that are greater than minimum peak height.
    mpd : positive integer, optional (default = 1)
        detect peaks that are at least separated by minimum peak distance (in
        number of data).
    threshold : positive number, optional (default = 0)
        detect peaks (valleys) that are greater (smaller) than `threshold`
        in relation to their immediate neighbors.
    edge : {None, 'rising', 'falling', 'both'}, optional (default = 'rising')
        for a flat peak, keep only the rising edge ('rising'), only the
        falling edge ('falling'), both edges ('both'), or don't detect a
        flat peak (None).
    kpsh : bool, optional (default = False)
        keep peaks with same height even if they are closer than `mpd`.
    valley : bool, optional (default = False)
        if True (1), detect valleys (local minima) instead of peaks.
    show : bool, optional (default = False)
        if True (1), plot data in matplotlib figure.
    ax : a matplotlib.axes.Axes instance, optional (default = None).

    Returns
    -------
    ind : 1D array_like
        indeces of the peaks in `x`.

    Notes
    -----
    The detection of valleys instead of peaks is performed internally by simply
    negating the data: `ind_valleys = detect_peaks(-x)`
    
    The function can handle NaN's 

    See this IPython Notebook [1]_.

    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb

    Examples
    --------
    >>> from detect_peaks import detect_peaks
    >>> x = np.random.randn(100)
    >>> x[60:81] = np.nan
    >>> # detect all peaks and plot data
    >>> ind = detect_peaks(x, show=True)
    >>> print(ind)

    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # set minimum peak height = 0 and minimum peak distance = 20
    >>> detect_peaks(x, mph=0, mpd=20, show=True)

    >>> x = [0, 1, 0, 2, 0, 3, 0, 2, 0, 1, 0]
    >>> # set minimum peak distance = 2
    >>> detect_peaks(x, mpd=2, show=True)

    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # detection of valleys instead of peaks
    >>> detect_peaks(x, mph=0, mpd=20, valley=True, show=True)

    >>> x = [0, 1, 1, 0, 1, 1, 0]
    >>> # detect both edges
    >>> detect_peaks(x, edge='both', show=True)

    >>> x = [-2, 1, -2, 2, 1, 1, 3, 0]
    >>> # set threshold = 2
    >>> detect_peaks(x, threshold = 2, show=True)
    """

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind)

    return ind

def _plot(x, mph, mpd, threshold, edge, valley, ax, ind):
    """Plot results of the detect_peaks function, see its help."""
    try:
        import matplotlib.pyplot as plt
    except ImportError:
        print('matplotlib is not available.')
    else:
        if ax is None:
            _, ax = plt.subplots(1, 1, figsize=(8, 4))

        ax.plot(x, 'b', lw=1)
        if ind.size:
            label = 'valley' if valley else 'peak'
            label = label + 's' if ind.size > 1 else label
            ax.plot(ind, x[ind], '+', mfc=None, mec='r', mew=2, ms=8,
                    label='%d %s' % (ind.size, label))
            ax.legend(loc='best', framealpha=.5, numpoints=1)
        ax.set_xlim(-.02*x.size, x.size*1.02-1)
        ymin, ymax = x[np.isfinite(x)].min(), x[np.isfinite(x)].max()
        yrange = ymax - ymin if ymax > ymin else 1
        ax.set_ylim(ymin - 0.1*yrange, ymax + 0.1*yrange)
        ax.set_xlabel('Data #', fontsize=14)
        ax.set_ylabel('Amplitude', fontsize=14)
        mode = 'Valley detection' if valley else 'Peak detection'
        ax.set_title("%s (mph=%s, mpd=%d, threshold=%s, edge='%s')"
                     % (mode, str(mph), mpd, str(threshold), edge))
        # plt.grid()
        plt.show()

def fit_max(x,y):
    #fit_r = fit_max_f.fit(data=y, x = x, x0 = x[np.argmax(y)],
      #                    a = np.amax(y), g = 100)
    return x[np.argmax(y)]
    return fit_r.params['x0'].value

def LorentzianLin(m,b,amp,xc,gam,x):
    outs=m*(x-xc)+b+(amp/(np.pi*2))/((x-xc)**2+(gam/2)**2)
    return outs

def Lorentzian(b,amp,xc,gam,x):
    outs=b+(amp/(np.pi*2))/((x-xc)**2+(gam/2)**2)
    return outs

def Quadratic(a,b,c,x):
    outs=a+b*x+c*x*x
    return outs

def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or np.fabs(value - array[idx-1]) < np.fabs(value - array[idx])):
        return idx-1
    else:
        return idx
    
def rollingWindow(a, window, edge = 'copy'):
    if edge == 'copy':
        extended = np.zeros(a.shape[:-1] + (a.shape[-1]+window-1, ))
        extended[..., int(window/2.-1): -int(window/2.)] = a[...]
        extended[..., :int(window/2.+1)]= a[...,0]
        extended[..., -int(window/2.):]= a[...,-1]
        a = extended
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def rollingAverage(a,win_size=10):
    out=average(rollingWindow(a, win_size), -1)
    out1=np.append(np.zeros(int(win_size/2)),out)
    out2=out1[:-int(win_size/2)] #these two modiciations make the filter symetric
    return out
    
import numpy as np
def find_nearest_unsrt(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [ ]:
test=detect_peaks(trans[1],mph=.12,mpd=5000,valley=True,threshold=-.1,show=True)

In [ ]:
peakrange=10 #number of peaks after a given one, we use to try and look for patterns
peaks=test[minpeak:maxpeak]
fwhm=[]
peaksize=[]
peakf=[]
peakdiffs=[[x for x in range(len(peaks))] for y in range(len(peaks))]
for i in range(len(peaks)): #gets peak size, location, and FWHM
    psize=trans[1,peaks[i]]
    pf=trans[0,peaks[i]]
    neghalf=find_nearest_unsrt(trans[1,peaks[i]-5000:peaks[i]],psize/2)
    poshalf=find_nearest_unsrt(trans[1,peaks[i]:peaks[i]+5000],psize/2)
    fwhmlocal=abs(trans[0,poshalf]-trans[0,neghalf])
    fwhm.append(fwhmlocal)
    peaksize.append(psize)
    peakf.append(pf)
    peakdiffs[i][:]=peaks-peaks[i]